In [19]:
import pandas as pd
import datasets
from datasets import load_dataset

batch = 0

# Load the dataset
data = pd.read_excel('outputs/main_data_batch_0_results.xlsx', sheet_name=None)

## remove the verifiability_extraction sheet
data.pop('verifiability_extraction')

,paper_id,venue,focused_review,review_point,id,chatgpt_verifiability_extraction_score,chatgpt_verifiability_extraction_rationale,prompt
0,NIPS_2022_511,NIPS_2022,1.Some theorems could be better formatted in t...,2. It would be helpful if the authors could fu...,8281,yes,The comment suggests that the authors should f...,\nThis aspect is aimed to maximize the utiliza...
1,ACL_2017_365_review,ACL_2017,1) Instead of arguing that the MTL approach re...,- the authors did not respond to my questions ...,354,yes,The comment suggests that the authors did not ...,\nThis aspect is aimed to maximize the utiliza...
2,diKRhKs5yl,ICLR_2025,1. Lack of experiments with larger-scale model...,"3. In the implementation of tree construction,...",11578,yes,The comment suggests that the implementation o...,\nThis aspect is aimed to maximize the utiliza...
3,tqHgSxRwiK,ICLR_2024,1. The premise and contribution of the paper i...,1. The premise and contribution of the paper i...,9274,yes,The review point expresses a lack of clarity r...,\nThis aspect is aimed to maximize the utiliza...
4,kBVPD2kJMy,ICLR_2025,Here are the weaknesses of this paper in my op...,2) Clarity and Accessibility: The paper is cha...,9647,yes,The review point contains claims about the cla...,\nThis aspect is aimed to maximize the utiliza...
...,...,...,...,...,...,...,...,...
995,UmdotAAVDe,ICLR_2025,1. As we know LLM often hallucinates. How do t...,1. As we know LLM often hallucinates. How do t...,11453,X,"The comment does not contain a claim, opinion,...",\nThis aspect is aimed to maximize the utiliza...
996,rWIrdAo2xC,ICLR_2025,"1. For GS dataset preparation, stage 1: per-sc...",4. The diffusion model design is very confusin...,9817,yes,"The comment claims that ""the diffusion model d...",\nThis aspect is aimed to maximize the utiliza...
997,JpyWPfzu0b,ICLR_2024,1) I strongly encourage authors to provide mor...,2) I understand the paper mainly focuses on a ...,11688,yes,The comment expresses an opinion and suggestio...,\nThis aspect is aimed to maximize the utiliza...
998,ACL_2017_387_review,ACL_2017,(1) Replicability would be an important concer...,"-General Discussion: Overall, this paper is we...",373,yes,The review point contains both positive feedba...,\nThis aspect is aimed to maximize the utiliza...


In [20]:
data['actionability'].columns

Index(['paper_id', 'venue', 'focused_review', 'review_point', 'id',
       'chatgpt_actionability_score', 'chatgpt_actionability_rationale',
       'prompt'],
      dtype='object')

In [21]:
### convert the df to dataset and store for each aspect
for aspect in data.keys():
    dataset = datasets.Dataset.from_pandas(data[aspect]).train_test_split(test_size=0.1)
    dataset.push_to_hub(f'boda/review_evaluation_automatic_labels',config_name=aspect )
    

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/3.80k [00:00<?, ?B/s]

...

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/3.80k [00:00<?, ?B/s]

In [22]:
merged_df = []

### aggregate the different aspects in one dataset
for i,row in data['actionability'].iterrows():
    row['actionbillity_prompt'] = row['prompt']
    ## remove the prompt column
    row.pop('prompt')

    for aspect in data.keys():
        if aspect == 'actionability':
            continue
        ### get the same row from the other aspects
        ## make sure the row exist
        if row['id'] not in data[aspect]['id'].values:
            print(f'row {row["id"]} not found in {aspect}')
            continue
        aspect_row = data[aspect][data[aspect]['id'] == row['id']].iloc[0]
        row[f'chatgpt_{aspect}_score'] = aspect_row[f'chatgpt_{aspect}_score']
        row[f'chatgpt_{aspect}_rationale'] = aspect_row[f'chatgpt_{aspect}_rationale']
        row[f'{aspect}_prompt'] = aspect_row['prompt']
    merged_df.append(row)

merged_df = pd.DataFrame(merged_df)

row 9881 not found in verifiability
row 5316 not found in verifiability


In [24]:
## convert the df to dataset and push to hub
dataset = datasets.Dataset.from_pandas(merged_df).train_test_split(test_size=0.1)
dataset.push_to_hub(f'boda/review_evaluation_automatic_labels',config_name='all' )

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/3.80k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/boda/review_evaluation_automatic_labels/commit/6301fb0be139cfe2a6b1a97b2bb19de23a3ad9cf', commit_message='Upload dataset', commit_description='', oid='6301fb0be139cfe2a6b1a97b2bb19de23a3ad9cf', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/boda/review_evaluation_automatic_labels', endpoint='https://huggingface.co', repo_type='dataset', repo_id='boda/review_evaluation_automatic_labels'), pr_revision=None, pr_num=None)